In [1]:
##################### Import Modules ############################
import modin.pandas as pmd
from umi_tools import UMIClusterer
from collections import defaultdict
import sys
import os
import json
import pandas as pd
from itertools import chain
import numpy as np
import ast
import sklearn.preprocessing as sk

In [7]:
read_dir='/media/jkbuntu/SAMSUNG2TB/Dropbox/KBJasim/Projects/Capture_Sequencing/LONG_TCR/FINAL_21_2/Repository/SCRIPTS/umi_correction.py'
os.path.basename(os.path.basename(read_dir)).split('.')[0]+'_'+datetime.now().strftime('%d_%m')

'umi_correction_27_06'

In [5]:
from datetime import datetime
datetime.now().strftime('%m_%d')

'2022_06_27'

In [8]:
sample_name='SPTCR16'
sample_name_extended='GBM275'

In [11]:
SPTCR=pmd.read_csv('/media/jkbuntu/SAMSUNG2TB/Dropbox/KBJasim/Projects/Capture_Sequencing/Spatial_Data/SPTCR_Demultiplexed/scTagger/UMI_Extraction/SPTCR_UMI_BARCODE_REVCOMP_CORRECTED/{0}_{1}_vdj_umi_barcode_revcomp_corrected.csv'.format(sample_name,sample_name_extended),index_col=0).dropna(subset=['locus','v_call','j_call','cdr3_aa'])

To remove this warning, run the following python code before doing dataframe operations:

    import ray
    ray.init()



In [17]:
read_dir='/media/jkbuntu/SAMSUNG2TB/Dropbox/KBJasim/Projects/Capture_Sequencing/Spatial_Data/SPTCR_Demultiplexed/scTagger/UMI_Extraction/SPTCR_UMI_BARCODE_REVCOMP_CORRECTED/SPTCR12_GBM259_vdj_umi_barcode_revcomp_corrected.csv'
sample_df=pmd.read_csv(read_dir)
sample_df

,ReadID,locus,v_call,d_call,j_call,cdr3,cdr3_aa,UMI,Indicator,Visium Barcode
0,7496867a-29a6-43a3-944c-8449fe35466b,TRA,TRAV1-1*01,NaN,TRAJ10*01,NaN,NaN,ATAATGTACGGG,Forward,TGGAACCACTGACACA
1,3e9af68d-e9c9-44ff-b624-10e9f1e3de66,TRA,TRAV1-1*01,NaN,TRAJ1*01,NaN,NaN,ATCGTACGACTT,Forward,TGGAACCACTGACACA
2,9c99ba0e-4b7e-4b37-b778-3635021a40fc,TRA,TRAV1-1*01,NaN,TRAJ1*01,NaN,NaN,ATCGTACGACTT,Forward,TGGAACCACTGACACA
3,149a0047-efdc-46ca-8283-2a398a65e4cc,TRA,TRAV11*01,NaN,TRAJ30*01,NaN,NaN,ATGTTGGTGAGG,Forward,TGGAACCACTGACACA
4,6b4641ae-959b-412c-bb1a-7dc7c85b3b01,TRA,TRAV14/DV4*01,NaN,TRAJ24*02,NaN,NaN,AGGATATCGCTT,Forward,TGGAACCACTGACACA
...,...,...,...,...,...,...,...,...,...,...
2000317,171ef913-37d4-4f22-aa55-8e16f0714837,TRA,TRBV21/OR9-2*01,NaN,TRAJ21*01,NaN,NaN,CCATACACCACT,Reverse Complement,TATAGCGCACGTTATC
2000318,814680a4-63f1-42ea-a2c6-ff4b0a7d531b,TRB,TRBV7-9*07,TRBD1*01,TRBJ2-5*01,NaN,NaN,TTTTTGACACAT,Reverse Complement,TAGTCCGTATGCATAA
2000319,7b5898b6-67fc-4d1e-a210-b0ce6051f157,TRB,TRBV7-9*01,TRBD1*01,TRBJ2-5*01,NaN,NaN,CATTTTGGGTCT,Reverse Complement,TCTCAGGCTACTCGCT
2000320,e264f3c3-48d3-4774-900e-ccd9a77163a2,TRB,TRBV7-9*01,TRBD2*02,TRBJ2-3*01,GCCAGCAGCTTACTGCCTAGCGGGAGAGGGATACGCAGTAT,ASSLLPSGRGIRS,AATTAGGGCCGG,Reverse Complement,GCTCGACCGAACTGAA


In [2]:
umi_bc='/media/jkbuntu/SAMSUNG2TB/Dropbox/KBJasim/Projects/Capture_Sequencing/LONG_TCR/FINAL_21_2/Repository/Pipeline/Test/Demultiplexing/Demultiplexing_test_demux/test_demux_barcode_umi.csv'
umi_bc_df=pd.read_csv(umi_bc)
umi_bc_df

,Spatial Barcode,ReadID,UMI
0,TGGGAGATACTTGTTT,0174f20e-b476-4ec3-a51f-a6895fcb846e,TTTGCAAGATTT
1,TGCTAGTAGATTGTTT,4224b8ac-ba13-49df-b862-96746132b926,TCACACCCGTTC
2,TGCTAGTAGATTGTTT,ec527ee2-49e7-43c1-bd07-b9a4425e22c5,TTTCGGGACGAT
3,TGCTAGTAGATTGTTT,753a2c95-b76d-46c6-b080-04fa591b7eab,GATTGTTTGTTT
4,TGCTAGTAGATTGTTT,37114111-fc84-4472-b06c-5bc67a85e615,GCGTGTCGCCAT
...,...,...,...
70249,TTGTTTGTGTAAATTC,6e7989c3-c0f9-4a84-a2be-1f2acb10e9ab,NaN
70250,TTGTTTGTGTAAATTC,019b6460-3086-4c8e-8397-41acfebdebaa,NaN
70251,TTGTTTGTGTAAATTC,d0add826-bc13-49d0-8946-976df94e1506,NaN
70252,TTGTTTGTGTAAATTC,d074404c-ec3b-446f-b6ea-1228a6d3e216,NaN


In [19]:
vdj=vdj.rename(columns={col_names[0]:'ReadID',col_names[1]:'Locus',col_names[2]:'V',col_names[3]:'D',col_names[4]:'J',col_names[5]:'CDR3',col_names[6]:'CDR3_aa'})

SyntaxError: invalid syntax (4042770967.py, line 1)

In [17]:
read_dir='/media/jkbuntu/SAMSUNG2TB/Dropbox/KBJasim/Projects/Capture_Sequencing/LONG_TCR/FINAL_21_2/Repository/Pipeline/Test/PreProcessing/IGB_Trimmed/test_demux_preprocessed_IGB.tsv'
sample_df=pd.read_csv(read_dir, sep='\t')
sample_df

,sequence_id,sequence,locus,stop_codon,vj_in_frame,v_frameshift,productive,rev_comp,complete_vdj,v_call,...,cdr3_start,cdr3_end,np1,np1_length,np2,np2_length,v_family,d_family,j_family,cdr3_aa_length
0,27:1278|f3c4d290-b93f-4c24-ab3d-173f8b509dd8ru...,GACAGACACAGTGCTGCCTGCCCCTTTGTGCCATGGGCTCCAGGCT...,TRB,T,F,T,F,T,F,TRBV5-1*01,...,362.0,399.0,CCTGAAGTGGCTTAGACACACT,22.0,NaN,NaN,TRBV5-1,NaN,TRBJ2-3,12
1,27:859|87f02d5d-5e89-4fc7-971f-2e5b2d77f196run...,TAATGTCTCTTGAACGGGGACCAAAGGGAGAGCCTACCCGGCAGAG...,TRB,T,F,F,F,T,F,TRBV3-1*01,...,NaN,NaN,TCATTAAGTTTTGCATA,17.0,NaN,NaN,TRBV3-1,NaN,TRBJ1-2,0
2,26:1257|c5e710cb-eefb-42b0-a02c-8f3a863bfac9ru...,CAGTGGTATCAAATTCATGGGGAGAGATCAGCAGAGTCATTGGGTT...,TRA,F,NaN,NaN,NaN,T,F,TRBV30*01,...,NaN,NaN,NaN,NaN,NaN,NaN,TRBV30,NaN,NaN,0
3,30:636|7cf5db8f-1794-4bdf-b3f0-bf9f9e8d9f72run...,GGTAGGTAATAAGATTTACTGAAAACGTCTCGGCCACATTCAGTAC...,TRB,T,F,F,F,F,F,TRBV6-8*01,...,129.0,220.0,TGTGTCCGTGCTG,13.0,GCCTCTGTCCAGGTGGTGAGTGTGGAACGGTGTGTGTGTGTGTCTG...,62.0,TRBV6-8,TRBD1,TRBJ1-4,31
4,26:1014|c24fd351-a4ec-4156-85dd-69957d5b3d93ru...,TTTTTTTTTTTTTTTTTTTTGGGTGCATTTGTTTGTATGTTAAAAA...,TRG,F,NaN,NaN,NaN,F,F,TRBV10-3*01,...,NaN,NaN,NaN,NaN,NaN,NaN,TRBV10-3,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67307,32:677|fbf8deb9-7fcd-4cdc-8c53-fb3c8f4c7b0crun...,GGGACCATATGGTGGAGGAAATCAGTGGTCCAGGAGGCAGTGGAGG...,TRB,F,T,F,T,T,F,TRBV12-2*01,...,NaN,NaN,TTTCTTAGCTTTTTTGTTCATGTTTTT,27.0,GGTCTTCTTTAGAGTGTAGAAGCATTCT,28.0,TRBV12-2,TRBD2,TRBJ1-1,0
67308,26:1140|87ce888b-77e9-4d95-ae7c-7a73e8a592acru...,GGTGGCGTAGAGCAAATTGCTGGAGCGGAAGTGACCGTGTGGCTTC...,TRA,T,NaN,NaN,F,T,F,TRBV6-8*01,...,NaN,NaN,NaN,NaN,NaN,NaN,TRBV6-8,NaN,NaN,0
67309,27:939|d74bf990-2bab-4ef1-b0f5-8abe409940efrun...,TTTATAGAAGCCTCCCTGGGCCAGCCTTTCTAGGAAAGCTGGAAAA...,TRB,T,F,F,F,F,F,TRBV21-1*02,...,NaN,NaN,AAAATGATGAAATCAACTTAAA,22.0,NaN,NaN,TRBV21-1,NaN,TRBJ2-1,0
67310,27:1060|c8b63b45-0515-44a8-8482-9b265e596599ru...,AGCTGCCCCCTCTGAGAGGGCTGCTGAGAGGCGCTGCTGGGCGTCT...,TRA,T,NaN,NaN,F,T,F,TRBV11-3*03,...,NaN,NaN,NaN,NaN,NaN,NaN,TRBV11-3,NaN,NaN,0


In [6]:
summary_sample_df=sample_df[['locus','v_call','d_call','j_call','cdr3','cdr3_aa','sequence_id']]
summary_sample_df

,locus,v_call,d_call,j_call,cdr3,cdr3_aa,sequence_id
0,TRB,TRBV5-1*01,NaN,TRBJ2-3*01,GCCAGCAGCCCTGAAGTGGCTTAGACACACTGCAGTAT,PAALKWLRHTAV,27:1278|f3c4d290-b93f-4c24-ab3d-173f8b509dd8ru...
1,TRB,TRBV3-1*01,NaN,TRBJ1-2*01,NaN,NaN,27:859|87f02d5d-5e89-4fc7-971f-2e5b2d77f196run...
2,TRA,TRBV30*01,NaN,NaN,NaN,NaN,26:1257|c5e710cb-eefb-42b0-a02c-8f3a863bfac9ru...
3,TRB,TRBV6-8*01,TRBD1*01,TRBJ1-4*01,GCCAGTGTGTCCGTGCTGGGGACGCCTCTGTCCAGGTGGTGAGTGT...,ASVSVLGTPLSRW*VWNGVCVCLCACYQRKT,30:636|7cf5db8f-1794-4bdf-b3f0-bf9f9e8d9f72run...
4,TRG,TRBV10-3*01,NaN,NaN,NaN,NaN,26:1014|c24fd351-a4ec-4156-85dd-69957d5b3d93ru...
...,...,...,...,...,...,...,...
67307,TRB,TRBV12-2*01,TRBD2*01,TRBJ1-1*01,NaN,NaN,32:677|fbf8deb9-7fcd-4cdc-8c53-fb3c8f4c7b0crun...
67308,TRA,TRBV6-8*01,NaN,NaN,NaN,NaN,26:1140|87ce888b-77e9-4d95-ae7c-7a73e8a592acru...
67309,TRB,TRBV21-1*02,NaN,TRBJ2-1*01,NaN,NaN,27:939|d74bf990-2bab-4ef1-b0f5-8abe409940efrun...
67310,TRA,TRBV11-3*03,NaN,NaN,NaN,NaN,27:1060|c8b63b45-0515-44a8-8482-9b265e596599ru...


In [7]:
summary_sample_df['sequence_id']=summary_sample_df['sequence_id'].str.split('|',n=1).str[1]
summary_sample_df['sequence_id']=summary_sample_df['sequence_id'].str.split('runid').str[0]
summary_sample_df

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,locus,v_call,d_call,j_call,cdr3,cdr3_aa,sequence_id
0,TRB,TRBV5-1*01,NaN,TRBJ2-3*01,GCCAGCAGCCCTGAAGTGGCTTAGACACACTGCAGTAT,PAALKWLRHTAV,f3c4d290-b93f-4c24-ab3d-173f8b509dd8
1,TRB,TRBV3-1*01,NaN,TRBJ1-2*01,NaN,NaN,87f02d5d-5e89-4fc7-971f-2e5b2d77f196
2,TRA,TRBV30*01,NaN,NaN,NaN,NaN,c5e710cb-eefb-42b0-a02c-8f3a863bfac9
3,TRB,TRBV6-8*01,TRBD1*01,TRBJ1-4*01,GCCAGTGTGTCCGTGCTGGGGACGCCTCTGTCCAGGTGGTGAGTGT...,ASVSVLGTPLSRW*VWNGVCVCLCACYQRKT,7cf5db8f-1794-4bdf-b3f0-bf9f9e8d9f72
4,TRG,TRBV10-3*01,NaN,NaN,NaN,NaN,c24fd351-a4ec-4156-85dd-69957d5b3d93
...,...,...,...,...,...,...,...
67307,TRB,TRBV12-2*01,TRBD2*01,TRBJ1-1*01,NaN,NaN,fbf8deb9-7fcd-4cdc-8c53-fb3c8f4c7b0c
67308,TRA,TRBV6-8*01,NaN,NaN,NaN,NaN,87ce888b-77e9-4d95-ae7c-7a73e8a592ac
67309,TRB,TRBV21-1*02,NaN,TRBJ2-1*01,NaN,NaN,d74bf990-2bab-4ef1-b0f5-8abe409940ef
67310,TRA,TRBV11-3*03,NaN,NaN,NaN,NaN,c8b63b45-0515-44a8-8482-9b265e596599


In [ ]:
python '/media/jkbuntu/SAMSUNG2TB/Dropbox/KBJasim/Projects/Capture_Sequencing/Github SPTCR-seq Pipeline/SPTCR-Seq-Pipeline/SCRIPTS/demultiplex_summarize.py' \
    -igb /media/jkbuntu/SAMSUNG2TB/Dropbox/KBJasim/Projects/Capture_Sequencing/LONG_TCR/FINAL_21_2/Repository/Pipeline/Test/PreProcessing/IGB_Trimmed/test_demux_preprocessed_IGB.tsv \
    -bc '/media/jkbuntu/SAMSUNG2TB/Dropbox/KBJasim/Projects/Capture_Sequencing/LONG_TCR/FINAL_21_2/Repository/Pipeline/Test/Demultiplexing/Demultiplexing_test_demux/test_demux_barcode_umi.csv'

In [ ]:
DEMULTIPLEXER="/media/jkbuntu/SAMSUNG2TB/Dropbox/KBJasim/Projects/Capture_Sequencing/Github SPTCR-seq Pipeline/SPTCR-Seq-Pipeline/SCRIPTS/demultiplex_summarize.py"

In [ ]:
python '/media/jkbuntu/SAMSUNG2TB/Dropbox/KBJasim/Projects/Capture_Sequencing/Github SPTCR-seq Pipeline/SPTCR-Seq-Pipeline/SCRIPTS/demultiplex_summarize.py' \
    -igb '/media/jkbuntu/SAMSUNG2TB/Dropbox/KBJasim/Projects/Capture_Sequencing/LONG_TCR/Testing_Zone/27.07/test_demux_preprocessed_IGB_27_06_vdj_umi_barcode_uncorrected_df.csv' \
    -bc '/media/jkbuntu/SAMSUNG2TB/Dropbox/KBJasim/Projects/Capture_Sequencing/LONG_TCR/FINAL_21_2/Repository/Pipeline/Test/Demultiplexing/Demultiplexing_test_demux/test_demux_barcode_umi.csv'

In [15]:
umi_bc=pd.read_csv('/media/jkbuntu/SAMSUNG2TB/Dropbox/KBJasim/Projects/Capture_Sequencing/LONG_TCR/FINAL_21_2/Repository/Pipeline/Test/Demultiplexing/Demultiplexing_test_demux/test_demux_barcode_umi.csv')

In [16]:
umi_bc

,Spatial Barcode,ReadID,UMI
0,TGGGAGATACTTGTTT,0174f20e-b476-4ec3-a51f-a6895fcb846e,TTTGCAAGATTT
1,TGCTAGTAGATTGTTT,4224b8ac-ba13-49df-b862-96746132b926,TCACACCCGTTC
2,TGCTAGTAGATTGTTT,ec527ee2-49e7-43c1-bd07-b9a4425e22c5,TTTCGGGACGAT
3,TGCTAGTAGATTGTTT,753a2c95-b76d-46c6-b080-04fa591b7eab,GATTGTTTGTTT
4,TGCTAGTAGATTGTTT,37114111-fc84-4472-b06c-5bc67a85e615,GCGTGTCGCCAT
...,...,...,...
70249,TTGTTTGTGTAAATTC,6e7989c3-c0f9-4a84-a2be-1f2acb10e9ab,NaN
70250,TTGTTTGTGTAAATTC,019b6460-3086-4c8e-8397-41acfebdebaa,NaN
70251,TTGTTTGTGTAAATTC,d0add826-bc13-49d0-8946-976df94e1506,NaN
70252,TTGTTTGTGTAAATTC,d074404c-ec3b-446f-b6ea-1228a6d3e216,NaN


In [9]:
demuxed=summary_sample_df.merge(umi_bc_df,left_on='sequence_id',right_on='ReadID')

In [13]:
demuxed=demuxed.drop('sequence_id',axis=1)

In [14]:
demuxed

,locus,v_call,d_call,j_call,cdr3,cdr3_aa,Spatial Barcode,ReadID,UMI
0,TRB,TRBV3-1*01,NaN,TRBJ1-2*01,NaN,NaN,CGGTTCAAGTAGGTGT,87f02d5d-5e89-4fc7-971f-2e5b2d77f196,NaN
1,TRA,TRBV30*01,NaN,NaN,NaN,NaN,TTCGCACTGTACGACA,c5e710cb-eefb-42b0-a02c-8f3a863bfac9,NaN
2,TRB,TRBV6-8*01,TRBD1*01,TRBJ1-4*01,GCCAGTGTGTCCGTGCTGGGGACGCCTCTGTCCAGGTGGTGAGTGT...,ASVSVLGTPLSRW*VWNGVCVCLCACYQRKT,AGGCCTATCAGGTACG,7cf5db8f-1794-4bdf-b3f0-bf9f9e8d9f72,AATCGTTGTGTT
3,TRG,TRBV10-3*01,NaN,NaN,NaN,NaN,ACGGCCAACATGGACT,c24fd351-a4ec-4156-85dd-69957d5b3d93,NaN
4,TRB,TRBV6-8*01,TRBD2*02,TRBJ2-7*01,NaN,NaN,TTGTTTCATTAGTCTA,3410d799-0234-4893-af81-2b254cc353a9,NaN
...,...,...,...,...,...,...,...,...,...
46726,NaN,NaN,NaN,NaN,NaN,NaN,AATGATGCGACTCCTG,31669558-5690-4f88-89ab-64b0344c2206,NaN
46727,TRB,TRBV9*01,NaN,TRBJ2-3*01,NaN,NaN,CAACTCCAACGTTTAG,ee596104-da44-4251-a98f-1fc17fad5e05,NaN
46728,TRA,TRBV17*02,NaN,NaN,NaN,NaN,GCACCGATCACCAAAC,ca248f1b-52dc-447a-a702-b890404d4180,ACGTGATGTGAT
46729,TRB,TRBV21-1*02,NaN,TRBJ2-1*01,NaN,NaN,TGTCAAACGCGCGGGC,d74bf990-2bab-4ef1-b0f5-8abe409940ef,GTTAGTTAGTTT


***

In [ ]:
umi_corr='/media/jkbuntu/SAMSUNG2TB/Dropbox/KBJasim/Projects/Capture_Sequencing/Github SPTCR-seq Pipeline/SPTCR-Seq-Pipeline/SCRIPTS/umi_correction.py'

umi_corr \ 
    -igb /media/jkbuntu/SAMSUNG2TB/Dropbox/KBJasim/Projects/Capture_Sequencing/LONG_TCR/FINAL_21_2/Repository/Pipeline/Test/PreProcessing/IGB_Trimmed/test_demux_preprocessed_IGB.tsv
    -u /media/jkbuntu/SAMSUNG2TB/Dropbox/KBJasim/Projects/Capture_Sequencing/LONG_TCR/FINAL_21_2/Repository/Pipeline/Test/PreProcessing/IGB_Trimmed/test_demux_preprocessed_IGB.tsv